In [37]:
import os 
from google import genai
from google.genai import types
from io import BytesIO
from PIL import Image as PILImage 
from dotenv import load_dotenv

## Configuration
- Loads API keys from .env
- Set Gemini client.
- Keeps output file name and model names as variables for flexibility. 

In [ ]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = gemini_api_key

gemini_client = genai.Client(api_key=gemini_api_key)

# Configuration 
output_file = "comic_image.png"
gemini_text_model = "gemini-2.0-flash"  # FREE for video analysis
gemini_image_model = "gemini-2.0-flash-preview-image-generation"  # FREE 500/day! #"gemini-2.5-flash-image"

## Video Analysis and Prompt Enhancement
- Takes a video URL and user's input description.
- Sends both to Gemini, asking it to enhance and rewrite the input as a single structured prompt for a comic strip .
- Returns the final enhanced prompt. 

In [53]:
def extract_comic_prompt_and_enhance(video_url, user_input):
    """Extract and enhance comic prompt using Gemini."""
    prompt = f"""
    You are an expert comic strip prompt creator. Your task is to generate a single, detailed ready-to-use prompt for an image generation AI. 
    You will use the user-provided panel descriptions and a video to create a complete prompt.

    ### User Inputs:
    {user_input}
    
    ### Your Task & Instructions:

    1. **Analyze and Enhance:** Watch the video at the provided URL: {video_url}. Based on the video's context, add relevant details to each of the four panel descriptions. Enhance the descriptions to make them more vivid and engaging.

    2. **Generate a Single Prompt:** Your entire output must be one single, cohesive prompt for the image generator.

    3. **Specify Format, Style, and Composition in the Final Prompt:** The final prompt you generate must explicitly instruct the image AI to create a comic strip with the following specifications:
       - **Layout:** A **4-panel** layout, arranged in a **2x2 grid**. Each panel must be **clearly framed and equally spaced**.
       - **Art Style:** A distinct **comic book art style only**. Strictly instruct for **no photorealism**.
       - **Text Elements:** Appropriate **dialogue in speech bubbles** and descriptive **caption text** for each panel.
       - **Compositional Integrity:** Ensure that **no part of the characters, speech bubbles, or dialogue is cropped or cut off** by the panel borders. All text inside speech bubbles must be **fully visible and legible**.

    4. **Set the Tone:** The comic must be **humorous**. Instruct the image AI to capture **exaggerated timing and over-the-top reactions**, in the style of classic internet memes.  

    5. **Content Moderation:**
       * You must strictly filter the final prompt to **remove any harmful or inappropriate content**.
       * **Handle Copyright-adjacent Material:**
            * If the user's input mentions a movie title, you **must remove the movie title** from your generated prompt. 
            * However, if a character name is mentioned, **keep the character name**. Your prompt should instruct the image AI to create a character that *resembles* the mentioned character, but is not an exact replica.
            * Replace any specific brands or logos with generic equivalents (e.g., "a soda can" instead of "Coca-Cola can").
            
    6. **Final Output:** Return only the final, complete prompt for the image AI without any of your own commentary, introductions, or extra text.
    """
    
    response = gemini_client.models.generate_content(
        model=gemini_text_model,
        contents=types.Content(
            parts=[
                types.Part(text=prompt),
                types.Part(
                    file_data=types.FileData(file_uri=video_url)
                )
            ]
        )
    )
    
    return response.text

## Main Generation Pipeline 


In [ ]:
def generate_with_gemini_free(prompt):
    """Generate comic image using FREE Gemini 2.0 Flash Image Generation."""
    print(f"🎨 Calling FREE Gemini 2.0 Flash Image (500/day limit)")
    print(f"   Prompt length: {len(prompt)} chars")
    
    # Generate image using FREE Gemini image generation
    response = gemini_client.models.generate_content(
        model=gemini_image_model,
        contents=[prompt],
        config=types.GenerateContentConfig(
            response_modalities=["TEXT", "IMAGE"]  
        )
    )
    
    # Extract image from response
    for part in response.candidates[0].content.parts:
        if part.inline_data is not None:
            return part.inline_data.data  # Returns bytes
        
    raise Exception("No image data in Gemini response")
def save_image(image_data, filename=None): 
    """Save the generated image to a file."""
    output_path = filename or output_file

    if isinstance(image_data, bytes):
        img = PILImage.open(BytesIO(image_data))
    else:   
        img = image_data

    img.save(output_path)
    print(f"✅ Comic saved as '{output_path}'")
    return output_path  

## Complete Comic Generation 

In [55]:
def generate_comic(video_url, user_input):
    """Complete comic generation pipeline using 100% FREE Gemini models."""

    # Step 1: Analyze video and enhance prompt (FREE)
    print("🔍 Analyzing video with FREE Gemini 2.0 Flash...")
    try:
        enhanced_prompt = extract_comic_prompt_and_enhance(video_url, user_input)
        print("✅ Prompt enhancement complete.")
        print(f"\n📝 Enhanced Prompt:\n{enhanced_prompt}\n")
    except Exception as e:
        print(f"❌ Failed to analyze video:")
        print(f"   Error: {str(e)}")
        return None

    # Step 2: Generate comic image (FREE - 500/day limit)
    print("🎨 Generating comic with FREE Gemini image generation...")
    try:
        image_data = generate_with_gemini_free(enhanced_prompt)
    except Exception as e:
        print(f"❌ Failed to generate image:")
        print(f"   Error type: {type(e).__name__}")
        print(f"   Error message: {str(e)}")
        
        # Check if it's a quota error
        if "429" in str(e) or "quota" in str(e).lower():
            print("\n⚠️  You may have hit the daily limit (500 images/day)")
            print("   Daily quota resets at midnight Pacific Time")
        
        return None

    # Step 3: Save the generated image
    print("💾 Saving the generated comic image...")
    output_path = save_image(image_data)
    return output_path

### Usage Example

In [56]:
# Example usage
video_url = "https://www.youtube.com/shorts/_q_NIQwwClc"
user_input = """  
Make the Girl a princess and the boy a knight.
"""

# Generate the comic
output_path = generate_comic(video_url, user_input)

if output_path:
    print(f"Comic generated and saved to: {output_path}")

    # Display the generated comic
    img = PILImage.open(output_path)
    img.show()  # Opens the image in default image viewer

else:
    print("Comic generation failed.")    

🔍 Analyzing video with FREE Gemini 2.0 Flash...
✅ Prompt enhancement complete.

📝 Enhanced Prompt:
Create a humorous 4-panel comic strip in a 2x2 grid layout, drawn in a distinct comic book art style (no photorealism). Each panel should be clearly framed and equally spaced. Ensure all elements within each panel, including characters, speech bubbles, and dialogue, are fully visible and not cropped. The dialogue must be legible. The comic should convey exaggerated timing and over-the-top reactions, reminiscent of classic internet memes.

Panel 1: A young princess, dressed in an elaborate, jewel-toned gown, stands demurely in a sunlit garden, holding a bouquet of flowers. Her expression is one of innocent joy. Caption: "The princess I was..." Speech bubble: "Oh, Sir Knight, your kindness overwhelms me!"

Panel 2: A grizzled knight in shining armor, helmet slightly askew, kneels before the princess, offering her a wilted daisy. A mud stain is visible on his armor. His expression is overly 

### Interactive Usage

In [46]:
# Interactive input version
video_url = input("Enter the video URL: ").strip()
user_input = input("Enter your comic description: ").strip()

if video_url and user_input:
    output_path = generate_comic(video_url, user_input)

    if output_path:
        print(f"Comic generated and saved to: {output_path}")

        # Display the generated comic
        img = PILImage.open(output_path)
        img.show()  # Opens the image in default image viewer

    else:
        print("Comic generation failed. Please provide both video URL and description.")

🔍 Analyzing video with FREE Gemini 2.0 Flash...
✅ Prompt enhancement complete.

📝 Enhanced Prompt:
```
Create a 4-panel comic strip, laid out in a 2x2 grid. The art style is classic comic book style only, no photorealism. Each panel should be clearly framed, equally spaced, and contain dialogue in speech bubbles and descriptive caption text. No part of the characters, speech bubbles, or dialogue should be cropped; all text must be fully visible and legible. The tone of the comic should be humorous, with exaggerated timing and over-the-top reactions, in the style of classic internet memes.

Panel 1:
Caption: "Year 2024. Rajveer has been waiting for his beloved Priya for what feels like an eternity."
A young man, resembling an Indian actor wearing a bright red turban in the style of the referenced song, sits on a park bench, looking expectantly at a digital smartwatch. He's dressed in modern clothing. Speech bubble: "She said 5 PM... It's 5:02! Where is she?"

Panel 2:
Caption: "Year 207